In [ ]:
# import 및 접속/로그인
from selenium import webdriver
from bs4 import BeautifulSoup as bs
from urllib.parse import quote
import pandas as pd
import time
import random
import os

driver = webdriver.Chrome('chromedriver.exe') # jpynd 파일과 같은 디렉토리에 있는 chromedriver.exe를 실행.
soup = bs(driver.page_source, 'html.parser')  # beautifulsoup 관련 프레임 변화 코드인 듯. 이해 부족
driver.implicitly_wait(3)                     # ..

driver.get('https://nid.naver.com/nidlogin.login') # 네이버 로그인 url을 접속하는 코드

driver.find_element_by_name('id').send_keys('수정1. 네이버 ID')       # 입력한 정보를 로그인 화면의 ID 란에 입력
driver.find_element_by_name('pw').send_keys('수정2. 네이버 PW') # 입력한 정보를 로그인 화면의 PW 란에 입력
driver.find_element_by_css_selector('#frmNIDLogin > fieldset > input').click() # 로그인 버튼 클릭

In [ ]:
base_url = '수정3. 카페 메인 주소'   ## 수정 필요 항목. 크롤링 하고 싶은 카페의 main 화면 html 주소를 복사
article_urls=[]   ## 게시글의 주소를 넣는 배열 초기화
how_many_page=int(input("몇 페이지를 크롤링하시겠습니까? : "))
                  ## how_many_page라는 변수를 이용하여 몇 개의 page를 크롤링 할 것인지 input
                  
for i in range(how_many_page):  ## 입력한 크롤링 페이지 수 만큼 반복

    driver.get(base_url+'수정4. 카페 페이지 url, 마지막 숫자는 {}으로 대체'.format(i+1))
                            ## 직접 추출할 수 있지만, 빠른 과제 수행을 위해 이렇게 처리하였습니다.
                            ## 코드 형식 상, 마지막에 숫자(1,2,3,4,...)가 기입되어 있는데, 이 부분만 {}으로 대체해줍니다.
                            ## 카페에서 키워드를 검색한 뒤 F12을 누르고, 페이지를 표시하는 하단의 숫자를 클릭하면 확인 가능합니다.

    css_selector = "input#topLayerQueryInput"
                  ## beautifulsoup을 이용한, 코드인 듯 한데, css에 대한 이해가 부족하여 완전히 이해하지는 못했습니다.
    search_inpupt_element = driver.find_element_by_css_selector("input#topLayerQueryInput")
                  ## 정보 추출을 위한, 프레임을 변화시키는 코드라고 추측합니다.
    iframe_element = driver.find_element_by_css_selector("iframe#cafe_main")
                  ## ...
    driver.switch_to_frame(iframe_element)
                  ## ...

    article_list = driver.find_elements_by_css_selector('span.aaa > a.m-tcol-c')
                  ## 키워드를 검색한 뒤, 나오는 15개의 게시글 목록의 주소를 크롤링합니다.
                  ## span.aaa > a.m-tcol-c를 이용하여 주소가 포함된 코드를 가져온 뒤
    article_urls = [ i.get_attribute('href') for i in article_list ]
                  ## get_attribute('href')를 이용하여, 코드에서 href 이름으로 저장되어 있는 게시글의 주소를
                  ## 초기화한 article_urls 배열에 저장합니다.

    i+=1                             ## 페이지 반복 조회를 위한 i+=1을 수행합니다.
    print("**crawlin page...%d" %i)  ## 몇 번째 page를 크롤링하는지 현황을 보여줍니다.
    num = 1                          ## 몇 번째 게시물인지 알기 위한 변수 초기화
    
    for article in article_urls:  ## article이라는 변수를 사용하여 
        driver.get(article)       ## article_urls 배열에 저장된 게시글에 하나씩 방문합니다.
                  
        driver.switch_to_frame('cafe_main')           ## beautifulsoup을 이용하여 article도 frame 변환을 해줍니다.
        soup = bs(driver.page_source, 'html.parser')  ## ...

        title = soup.select('div.tit-box span.b')[0].get_text() ## 게시글의 제목을 title이라는 변수로 저장합니다.
        content_tags = soup.select('#tbody')[0].select('p')     
                  ## #tbody-p로 분류되어 있는 게시글 내용을 content_tags라는 배열로 가져옵니다.
        content = ' '.join([ tags.get_text() for tags in content_tags ])
                  ## 게시글 내용 전체를 띄어쓰기 하나로 구분하여 전체 게시글로 합칩니다.
                  ## 이는 하나의 게시글 안에 여러 문단으로 떨어져 있는 게시글을 하나의 변수로 저장하는 역할을 수행합니다.

        reply_tags = soup.select('#cmt_list')[0].select('span.comm_body')  ## 댓글 목록을 저장하는 역할을 수행합니다.
        reply = ' '.join([tags.get_text() for tags in reply_tags ]) ## 전체 댓글은 reply라는 변수를 이용하여 하나의 변수로 저장합니다.
        
        print('%d..' %num, end=' ')  ## page 내에서 몇 번째 게시글인지 표시합니다.
        print(title, end=" : ")      ## 제목을 표시합니다.
        print(content)               ## 게시글 내용을 표시합니다.
        print(reply)                 ## 게시글 댓글을 표시합니다.
        num += 1                     ## 페이지 내의 게시글 반복 조회를 위한 num+=1을 수행합니다.
    print('\n')